
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

# Step 1: Define your configurations

In [20]:
%stop_session

Stopping session: 2dab248f-060f-4bd5-aa02-582c282ace51
Stopped session.


In [36]:
%connections hudi-connection
%glue_version 3.0
%region us-east-1
%worker_type G.1X
%number_of_workers 2
%spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
%additional_python_modules Faker

Connections to be included:
hudi-connection
Setting Glue version to: 3.0
Previous region: us-east-1
Setting new region to: us-east-1
Reauthenticating Glue client with new region: us-east-1
IAM role has been set to arn:aws:iam::043916019468:role/Lab3. Reauthenticating.
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::043916019468:role/Lab3
Authentication done.
Region is set to: us-east-1
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 2
Setting new number of workers to: 2
Previous Spark configuration: spark.serializer=org.apache.spark.serializer.KryoSerializer
Setting new Spark configuration to: spark.serializer=org.apache.spark.serializer.KryoSerializer
Additional python modules to be included:
Faker


# Step 2: Define your Imports

In [1]:
try:
    import sys
    from awsglue.transforms import *
    from awsglue.utils import getResolvedOptions
    from pyspark.context import SparkContext
    from awsglue.context import GlueContext
    from awsglue.job import Job
    from pyspark.sql.session import SparkSession
    from awsglue.dynamicframe import DynamicFrame
    from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when
    from pyspark.sql.functions import *
    from awsglue.utils import getResolvedOptions
    from pyspark.sql.types import *
    from datetime import datetime
    import boto3
    from functools import reduce
    from datetime import datetime, timezone
    from random import randint
    import datetime
    from faker import Faker
    import random

except Exception as e:
    pass

Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 82c403ad-9a28-4877-a969-a194495f7be4
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--additional-python-modules Faker
Waiting for session 82c403ad-9a28-4877-a969-a194495f7be4 to get into ready status...
Session 82c403ad-9a28-4877-a969-a194495f7be4 has been created.



# Step 3: Create Spark Session

In [2]:
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').config('spark.sql.legacy.pathOptionBehavior.enabled', 'true').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
logger = glueContext.get_logger()

# Step 4: Read Data from S3 and Performing Upsert on Datalake 
### To practise Concepts we shall generate fake data and work off that 

In [4]:
RAW_SOURCE= "s3://glue-learn-begineers/topics/postgres.public.sales/"

In [5]:
AmazonS3_node1672188803752 = glueContext.create_dynamic_frame.from_options(
    format_options={"multiline": False},
    connection_type="s3",
    format="json",
    connection_options={
        "paths": [RAW_SOURCE],
        "recurse": True,
    },
    transformation_ctx="AmazonS3_node1672188803752",
)

In [6]:
spark_df = AmazonS3_node1672188803752.toDF()

In [7]:
spark_df.show(3)

+------+--------------------+--------------------+---+-------------+
|before|               after|              source| op|        ts_ms|
+------+--------------------+--------------------+---+-------------+
|  null|{19183, 98, Kitch...|{0.10.0.Final, po...|  c|1672192114302|
|  null|{6843, 2, Kitchen...|{0.10.0.Final, po...|  c|1672192116311|
|  null|{17175, 39, Offic...|{0.10.0.Final, po...|  c|1672192118332|
+------+--------------------+--------------------+---+-------------+
only showing top 3 rows


In [8]:
spark_df.select("after.*").show(truncate=False)

+---------+------+---------+---------+--------+----------------+----------------+------------+----------------+
|invoiceid|itemid|category |price    |quantity|orderdate       |destinationstate|shippingtype|referral        |
+---------+------+---------+---------+--------+----------------+----------------+------------+----------------+
|19183    |98    |Kitchen  |{0, FQ==}|3       |1482451200000000|MT              |2-Day       |Online Ad       |
|6843     |2     |Kitchen  |{0, JQ==}|1       |1460678400000000|OK              |2-Day       |Friend/Colleague|
|17175    |39    |Office   |{0, Og==}|1       |1460419200000000|ME              |Free        |Repeat Customer |
|18975    |62    |Kitchen  |{0, Mw==}|4       |1471737600000000|OR              |2-Day       |Other           |
|6129     |43    |Office   |{0, LQ==}|2       |1457049600000000|OH              |3-Day       |Friend/Colleague|
|10153    |54    |Garden   |{0, Tw==}|3       |1468886400000000|WY              |Free        |Repeat Cus

In [9]:
spark_df = spark_df.select("after.*")

In [11]:
spark_df = spark_df.drop("price")

In [12]:
spark_df.show(1)

+---------+------+--------+--------+----------------+----------------+------------+---------+
|invoiceid|itemid|category|quantity|       orderdate|destinationstate|shippingtype| referral|
+---------+------+--------+--------+----------------+----------------+------------+---------+
|    19183|    98| Kitchen|       3|1482451200000000|              MT|       2-Day|Online Ad|
+---------+------+--------+--------+----------------+----------------+------------+---------+
only showing top 1 row


# Step 5: Define Hudi Settings 

In [13]:
db_name = "hudidb"
table_name="sales"

recordkey = 'invoiceid'
precombine = 'itemid'

path = f"s3://glue-learn-begineers/hudi/{table_name}/"
method = 'upsert'                                 # bulk_insert | upsert | insert | delete
table_type = "COPY_ON_WRITE"

curr_session = boto3.session.Session()
curr_region = curr_session.region_name


In [14]:
connection_options={
    "path": path,
    "connectionName": "hudi-connection",

    "hoodie.datasource.write.storage.type": table_type,
    'className': 'org.apache.hudi',
    'hoodie.table.name': table_name,
    'hoodie.datasource.write.recordkey.field': recordkey,
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': method,
    'hoodie.datasource.write.precombine.field': precombine,


    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': db_name,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',
    
    
    
    'hoodie.write.concurrency.mode' : 'optimistic_concurrency_control'
    ,'hoodie.cleaner.policy.failed.writes' : 'LAZY'
    ,'hoodie.write.lock.provider' : 'org.apache.hudi.aws.transaction.lock.DynamoDBBasedLockProvider'
    ,'hoodie.write.lock.dynamodb.table' : 'hudi-lock-table'
    ,'hoodie.write.lock.dynamodb.partition_key' : 'tablename'
    ,'hoodie.write.lock.dynamodb.region' : '{0}'.format(curr_region)
    ,'hoodie.write.lock.dynamodb.endpoint_url' : 'dynamodb.{0}.amazonaws.com'.format(curr_region)
    ,'hoodie.write.lock.dynamodb.billing_mode' : 'PAY_PER_REQUEST'
    ,'hoodie.bulkinsert.shuffle.parallelism': 2000
    
    
    
}

# Step 5: Write data into HUDI Table

In [15]:
import datetime 
start = datetime.datetime.now()
ApacheHudiConnector0101forAWSGlue30_node1671045598524 = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)
end = datetime.datetime.now()
print("Execution Time: {} ".format(end-start))

Execution Time: 0:00:22.547273
